# Libraries

In [ ]:
# Libraries

import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')

!pip install ktrain
import keras
import ktrain
from ktrain import text

# Functions

In [ ]:
# test dataset - tweets - the sentences are merged into a single tweet

def dataframe_full_text(dataframe):
  del dataframe['Unnamed: 0']
  del dataframe['sentence']
  lines, columns = dataframe.shape

  if columns == 2:
    df_final_test = dataframe
    df_final_test.columns = ['sentence','target']

  if columns > 2:
    target = dataframe['target']
    del dataframe['target']
    lines, columns = dataframe.shape
    array_test = np.array(dataframe)
    array_empty = []
    
    for line in range(lines):
      final_list = []
      for column in range(0,columns-1,2):
        string = str(array_test[line][column])
        if string != '' and string != '??' and string != 'nan':
          if '1.0' in string:
            string = re.sub(r'1.0','1',string)
          if '2.0' in string:
            string = re.sub(r'2.0','2',string)
          if '3.0' in string:
            string = re.sub(r'3.0','3',string)
          if '4.0' in string:
            string = re.sub(r'4.0','4',string)
          if '5.0' in string:
            string = re.sub(r'5.0','5',string)
          final_list.append(string)
      final_string = ' '.join(final_list)
      array_empty.append(final_string)

    df_string_concat = pd.DataFrame(array_empty)
    df_final_test = pd.concat((df_string_concat, target), axis = 1)
    df_final_test.columns = ['sentence','target']

  return df_final_test

In [ ]:
# test dataset - sentences - the dataset comprises only sentences

def dataframe_sentences(dataframe):
  del dataframe['Unnamed: 0']
  del dataframe['sentence']
  lines, columns = dataframe.shape

  if columns == 2:
    df = dataframe

  if columns > 2:
    del dataframe['target']
    lines, columns = dataframe.shape
    array_test = np.array(dataframe)
    sentence_target_list = []
    
    for line in range(lines):
      for column in range(0,columns):
        string = str(array_test[line][column])

        if string != '' and string != '??' and string != 'nan' and string != 'NaN':
          if ((len(sentence_target_list) != 0) and ((sentence_target_list[-1] == '1') or (sentence_target_list[-1] == '2') or (sentence_target_list[-1] == '3') or (sentence_target_list[-1] == '4') or (sentence_target_list[-1] == '5')) and ((string != '1') or (string != '2') or (string != '3') or (string != '4') or (string != '5'))):
            if '1.0' in string:
              string = re.sub(r'1.0','1',string)
            if '2.0' in string:
              string = re.sub(r'2.0','2',string)
            if '3.0' in string:
              string = re.sub(r'3.0','3',string)
            if '4.0' in string:
              string = re.sub(r'4.0','4',string)
            if '5.0' in string:
              string = re.sub(r'5.0','5',string)
            sentence_target_list.append(string)
          if ((len(sentence_target_list) != 0) and ((sentence_target_list[-1] != '1') or (sentence_target_list[-1] != '2') or (sentence_target_list[-1] != '3') or (sentence_target_list[-1] != '4') or (sentence_target_list[-1] != '5')) and ((string == '1') or (string == '2') or (string == '3') or (string == '4') or (string == '5'))):
            if '1.0' in string:
              string = re.sub(r'1.0','1',string)
            if '2.0' in string:
              string = re.sub(r'2.0','2',string)
            if '3.0' in string:
              string = re.sub(r'3.0','3',string)
            if '4.0' in string:
              string = re.sub(r'4.0','4',string)
            if '5.0' in string:
              string = re.sub(r'5.0','5',string)
            sentence_target_list.append(string)
          if len(sentence_target_list) == 0:
            sentence_target_list.append(string)

    length = len(sentence_target_list)
    lines = int(length/2)
    columns = 2
    list_to_array = np.array(sentence_target_list)
    final_array = np.reshape(list_to_array, (lines, columns))
    df = pd.DataFrame(final_array)
  df.columns = ['sentence','target']

  return df

In [ ]:
# classification - one-hot

def one_hot(dataframe):
  list_01 = [1,0,0,0,0]
  list_02 = [0,1,0,0,0]
  list_03 = [0,0,1,0,0]
  list_04 = [0,0,0,1,0]
  list_05 = [0,0,0,0,1]
  lines, columns = dataframe.shape
  df_array = np.array(dataframe)
  one_hot_vectors = []

  for i in range(lines):
    if df_array[i][1] == 1 or df_array[i][1] == 1.0:
      one_hot_vectors.append(list_01)
    elif df_array[i][1] == 2 or df_array[i][1] == 2.0:
      one_hot_vectors.append(list_02)
    elif df_array[i][1] == 3 or df_array[i][1] == 3.0:
      one_hot_vectors.append(list_03)
    elif df_array[i][1] == 4 or df_array[i][1] == 4.0:
      one_hot_vectors.append(list_04)
    elif df_array[i][1] == 5 or df_array[i][1] == 5.0:
      one_hot_vectors.append(list_05)

  one_hot_array = np.array(one_hot_vectors).reshape(len(one_hot_vectors),5)
  one_hot_dataframe = pd.DataFrame(one_hot_array)
  del dataframe['target']
  df_one_hot = pd.concat((dataframe,one_hot_dataframe), axis = 1)
  df_one_hot.columns = ['sentence', '01','02','03','04','05']

  return df_one_hot

# Loading files

In [ ]:
# Loading files to dataframe

# train dataset
df_train = one_hot(dataframe_sentences(pd.read_excel('train_bert.xlsx')))

# test dataset
df_test_sentence = one_hot(dataframe_sentences(pd.read_excel('test_bert.xlsx')))
df_test_tweet = one_hot(dataframe_full_text(pd.read_excel('test_bert.xlsx')))

# Test 01

In [ ]:
# Create vectors https://github.com/amaiya/ktrain/blob/master/ktrain/text/data.py
(train_data_00), (test_data_00), preproc = text.texts_from_df(df_train, text_column='sentence',
                                                                                      label_columns=['01','02','03','04','05'],
                                                                                      maxlen=64,
                                                                                      preprocess_mode = 'distilbert',
                                                                                      lang = None,
                                                                                      ngram_range=1,
                                                                                      val_df = df_test_sentence,
                                                                                      random_state = 42,
                                                                                      )


preprocessing train...
language: pt
train sequence lengths:
	mean : 10
	95percentile : 27
	99percentile : 39


Is Multi-Label? False
preprocessing test...
language: pt
test sequence lengths:
	mean : 8
	95percentile : 22
	99percentile : 33


In [ ]:
learn_distilbert_test_01 = ktrain.get_learner(text.text_classifier('distilbert', (train_data_00) , preproc=preproc), 
                             train_data=(train_data_00), 
                             val_data=(test_data_00), batch_size = 5
                             )

learn_distilbert_test_01.fit_onecycle(0.01,3)

learn_distilbert_test_01.validate()

Is Multi-Label? False
maxlen is 64



done.


begin training using onecycle policy with max lr of 0.01...
Epoch 1/3
234/234 [==============================] - 664s 3s/step - loss: 1.3273 - accuracy: 0.3573 - val_loss: -95760197206453256192.0000 - val_accuracy: 0.0000e+00
Epoch 2/3
234/234 [==============================] - 667s 3s/step - loss: 1.3904 - accuracy: 0.3368 - val_loss: -84692873801073950720.0000 - val_accuracy: 0.0000e+00
Epoch 3/3
234/234 [==============================] - 666s 3s/step - loss: 1.2490 - accuracy: 0.3419 - val_loss: -91717363319865081856.0000 - val_accuracy: 0.0000e+00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     218.0
           3       0.00      0.00      0.00       0.0

    accuracy                           0.00     218.0
   macro avg       0.00      0.00      0.00     218.0
weighted avg       0.00      0.00      0.00     218.0



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  0, 218],
       [  0,   0]])

# Test 02

In [ ]:
(train_data_01), (test_data_01), preproc01 = text.texts_from_df(df_train, text_column = 'sentence',
                                                                                    label_columns = ['01','02','03','04','05'],
                                                                                    maxlen = 64, 
                                                                                    preprocess_mode = 'distilbert',
                                                                                    lang = None,
                                                                                    ngram_range = 1,
                                                                                    val_df = df_test_tweet,
                                                                                    random_state = 42
                                                                                    )

learn_distilbert_test_02 = ktrain.get_learner(text.text_classifier('distilbert', (train_data_01) , preproc = preproc01), 
                             train_data = (train_data_01), 
                             val_data = (test_data_01), batch_size = 10
                             )

learn_distilbert_test_02.fit_onecycle(0.01,3)

learn_distilbert_test_02.validate()

preprocessing train...
language: pt
train sequence lengths:
	mean : 10
	95percentile : 27
	99percentile : 39


Is Multi-Label? False
preprocessing test...
language: pt
test sequence lengths:
	mean : 18
	95percentile : 42
	99percentile : 49


Is Multi-Label? False
maxlen is 64
done.


begin training using onecycle policy with max lr of 0.01...
Epoch 1/3
117/117 [==============================] - 518s 4s/step - loss: 1.3269 - accuracy: 0.3308 - val_loss: 1.5319 - val_accuracy: 0.0780
Epoch 2/3
117/117 [==============================] - 516s 4s/step - loss: 1.2841 - accuracy: 0.3479 - val_loss: 1.4342 - val_accuracy: 0.4358
Epoch 3/3
117/117 [==============================] - 518s 4s/step - loss: 1.2383 - accuracy: 0.3513 - val_loss: 1.4967 - val_accuracy: 0.4358
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        80
           2       0.00      0.00      0.00        17
           3       0.44      1.00      0.61        95
           4       0.00      0.00      0.00        25

    accuracy                           0.44       218
   macro avg       0.09      0.20      0.12       218
weighted avg       0.19      0.44    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 0,  0,  0,  1,  0],
       [ 0,  0,  0, 80,  0],
       [ 0,  0,  0, 17,  0],
       [ 0,  0,  0, 95,  0],
       [ 0,  0,  0, 25,  0]])

# Load files

In [ ]:
# Loading files to dataframe

# train dataset
df_train = one_hot(dataframe_sentences(pd.read_excel('train_bag.xlsx')))

# test dataset
df_test_sentence = one_hot(dataframe_sentences(pd.read_excel('test_bag.xlsx')))
df_test_tweet = one_hot(dataframe_full_text(pd.read_excel('test_bag.xlsx')))

# Test 03

In [ ]:
# Create vectors https://github.com/amaiya/ktrain/blob/master/ktrain/text/data.py
(train_data_00), (test_data_00), preproc = text.texts_from_df(df_train, text_column='sentence',
                                                                                      label_columns=['01','02','03','04','05'],
                                                                                      maxlen=64,
                                                                                      preprocess_mode = 'distilbert',
                                                                                      lang = None,
                                                                                      ngram_range=1,
                                                                                      val_df = df_test_sentence,
                                                                                      random_state = 42,
                                                                                      )


preprocessing train...
language: pt
train sequence lengths:
	mean : 6
	95percentile : 16
	99percentile : 23


Is Multi-Label? False
preprocessing test...
language: pt
test sequence lengths:
	mean : 5
	95percentile : 13
	99percentile : 19


In [ ]:
learn_distilbert_test_01 = ktrain.get_learner(text.text_classifier('distilbert', (train_data_00) , preproc=preproc), 
                             train_data=(train_data_00), 
                             val_data=(test_data_00), batch_size = 5
                             )

learn_distilbert_test_01.fit_onecycle(0.01,3)

learn_distilbert_test_01.validate()

Is Multi-Label? False
maxlen is 64



done.


begin training using onecycle policy with max lr of 0.01...
Epoch 1/3
234/234 [==============================] - 645s 3s/step - loss: 1.2981 - accuracy: 0.3624 - val_loss: -160613808651378884608.0000 - val_accuracy: 0.0000e+00
Epoch 2/3
234/234 [==============================] - 645s 3s/step - loss: 1.3111 - accuracy: 0.3385 - val_loss: -117635148166824198144.0000 - val_accuracy: 0.0000e+00
Epoch 3/3
234/234 [==============================] - 645s 3s/step - loss: 1.2301 - accuracy: 0.3530 - val_loss: -110776245099176132608.0000 - val_accuracy: 0.0000e+00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     218.0
           2       0.00      0.00      0.00       0.0

    accuracy                           0.00     218.0
   macro avg       0.00      0.00      0.00     218.0
weighted avg       0.00      0.00      0.00     218.0



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  0, 218],
       [  0,   0]])

# Test 04

In [ ]:
(train_data_01), (test_data_01), preproc01 = text.texts_from_df(df_train, text_column = 'sentence',
                                                                                    label_columns = ['01','02','03','04','05'],
                                                                                    maxlen = 64, 
                                                                                    preprocess_mode = 'distilbert',
                                                                                    lang = None,
                                                                                    ngram_range = 1,
                                                                                    val_df = df_test_tweet,
                                                                                    random_state = 42
                                                                                    )

learn_distilbert_test_02 = ktrain.get_learner(text.text_classifier('distilbert', (train_data_01) , preproc = preproc01), 
                             train_data = (train_data_01), 
                             val_data = (test_data_01), batch_size = 10
                             )

learn_distilbert_test_02.fit_onecycle(0.01,3)

learn_distilbert_test_02.validate()

preprocessing train...
language: pt
train sequence lengths:
	mean : 6
	95percentile : 16
	99percentile : 23


Is Multi-Label? False
preprocessing test...
language: pt
test sequence lengths:
	mean : 11
	95percentile : 25
	99percentile : 31


Is Multi-Label? False
maxlen is 64
done.


begin training using onecycle policy with max lr of 0.01...
Epoch 1/3
117/117 [==============================] - 506s 4s/step - loss: 1.3511 - accuracy: 0.3308 - val_loss: 1.3632 - val_accuracy: 0.4358
Epoch 2/3
117/117 [==============================] - 501s 4s/step - loss: 1.2639 - accuracy: 0.3376 - val_loss: 1.6550 - val_accuracy: 0.0780
Epoch 3/3
117/117 [==============================] - 501s 4s/step - loss: 1.2413 - accuracy: 0.3581 - val_loss: 1.4909 - val_accuracy: 0.0780
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        80
           2       0.08      1.00      0.14        17
           3       0.00      0.00      0.00        95
           4       0.00      0.00      0.00        25

    accuracy                           0.08       218
   macro avg       0.02      0.20      0.03       218
weighted avg       0.01      0.08    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[ 0,  0,  1,  0,  0],
       [ 0,  0, 80,  0,  0],
       [ 0,  0, 17,  0,  0],
       [ 0,  0, 95,  0,  0],
       [ 0,  0, 25,  0,  0]])